In [1]:
import sys
# sys.path.append('/Users/joachim/texjs/lva/IntroSC/ASC-ODE/build/mechsystem')
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [2]:
import numpy as np
from mass_spring import *

mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)

L1 = 1.0   
L2 = 1.0  
f1 = mss.add(Fix((0.0, 0.0, 0.0)))

theta1 = 0.6  
theta2 = 0.9   

p1 = np.array([ L1*np.sin(theta1), 0.0, -L1*np.cos(theta1) ], dtype=float)
p2 = p1 + np.array([ L2*np.sin(theta2), 0.0, -L2*np.cos(theta2) ], dtype=float)

v1 = (0.0, 0.4, 0.0)
v2 = (0.0, 0.4, 0.0)

m1 = mss.add(Mass(1.0, tuple(p1), v1))
m2 = mss.add(Mass(1.0, tuple(p2), v2))

mss.add(DistanceConstraint(f1, m1, float(np.linalg.norm(p1 - np.array([0.0,0.0,0.0]))))) 
mss.add(DistanceConstraint(m1, m2, float(np.linalg.norm(p2 - p1))))                    

print("masses:", len(mss.masses), "joints:", len(mss.joints))


masses: 2 joints: 2


In [3]:
from pythreejs import *
from time import sleep

mass_meshes = []
for m in mss.masses:
    mass_meshes.append(
        Mesh(
            SphereBufferGeometry(0.18, 16, 16),
            MeshStandardMaterial(color="red"),
            position=(float(m.pos[0]), float(m.pos[1]), float(m.pos[2])),
        )
    )

fix_meshes = []
for f in mss.fixes:
    fix_meshes.append(
        Mesh(
            SphereBufferGeometry(0.20, 16, 16),
            MeshStandardMaterial(color="blue"),
            position=(float(f.pos[0]), float(f.pos[1]), float(f.pos[2])),
        )
    )

def joint_segments():
    segs = []
    for j in mss.joints:
        pA = mss[j.connectors[0]].pos
        pB = mss[j.connectors[1]].pos
        segs.append([[float(pA[0]), float(pA[1]), float(pA[2])],
                     [float(pB[0]), float(pB[1]), float(pB[2])]])
    return segs

j_geo = LineSegmentsGeometry(positions=joint_segments())
j_mat = LineMaterial(linewidth=4, color="cyan")
j_line = LineSegments2(j_geo, j_mat)

axes = AxesHelper(1)

view_width = 700
view_height = 450

camera = PerspectiveCamera(position=[3, 2, 2], aspect=view_width/view_height)
key_light = DirectionalLight(position=[5, 10, 5], intensity=0.8)
ambient_light = AmbientLight(intensity=0.6)

scene = Scene(children=[*mass_meshes, *fix_meshes, j_line, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=view_width, height=view_height)

renderer


Renderer(camera=PerspectiveCamera(aspect=1.5555555555555556, position=(3.0, 2.0, 2.0), projectionMatrix=(1.0, …

In [4]:
from pythreejs import LineSegmentsGeometry
from time import sleep

for i in range(3000):
    mss.simulate(0.01, 30)

    for m, mesh in zip(mss.masses, mass_meshes):
        mesh.position = (float(m.pos[0]), float(m.pos[1]), float(m.pos[2]))

    j_line.geometry = LineSegmentsGeometry(positions=joint_segments())

    sleep(0.01)
